# 1. CONSTRUCCION DEL DATASET

## Se construirá un dataset de muestra de los sellers de todas las categorías disponibles en Mercado Libre Argentina
#### Construccion del dataset
Para ello se tomaron aplicará la siguiente metodología:
1. El listado de las categorías, con su correspondiente id. Se trabajará con todas las categorias excepto las que en la exploracion tienen faltantes de datos de interes
2. Se buscarán los primeros 1000 items de dichas categorias y se extraerán datos de los sellers  
    esto nos permitirá obtener una muestra de sellers de distintos rubros y categorias, se buscó bajar la influencia de las publicaciones más relevantes
    y al quitar duplicados de sellers, se tendrá un listado de candidatos para **identificar a aquellos que tienen un buen perfil y son relevantes para el negocio**
3. Se seleccionaron los siguientes datos de los sellers dentro de cada item:
    - Cancelaciones, Reclamos y Demoras de los últimos 60 días (cantidad y porcentaje del total de operaciones)
    - El total de transacciones históricas: canceladas y completas
    - Rating de los usuarios: negativo, neutral y positivos
    - Precio de venta y ventas del item
    - Provincia y Localidad
4. Es relevante para el negovio contar con información acerca de la facturación de los sellers: Por lo tanto, se estimará a partir de los items obtenidos
    un aproximado de facturacion, se multiplicará la cantidad vendida por su precio, con esto se espera obtener una metrica de facturacion
    y, con esto, calcular el valor del ticket promedio (al sumarizar el total facturado y dividirlo por las ventas de cada seller) 
    y una metrica aproximada de la ganancia del negocio por cada seller, al calcular un % de ganancia sobre el total facturado. 
    Los datos se agruparán por seller para continuar trabajando con dicha unidad de análisis. 

Estás decisiones se basan en la hipotesis de que los sellers con mejores metricas, ratings, transacciones y rentabilidad serán los relevantes y tengan un buen perfil para el negocio
Respecto de la construccion del dataset, se indagó en la documentación de la API y en sus resultados y se optó por esta combinacion de consultas para llegar al dataset final

#### Preguntas
¿Qué datos se encuentran disponibles en la API de items? ¿Cuales son de los items, de los sellers o del negocio?
¿Qué datos se pueden conseguir para la clasificación de los sellers?
¿Es posible obtener el listado total de items que tiene el seller?
Al obtener datos por item, ¿Cómo sumarizar para obtener metricas por seller? Sumadas a las que indican su reputacion
¿Qué categorias utilizar para el análisis?


#### Idea
- Al quitar duplicados de sellers, buscar todos los items activos para obtener el total de publicaciones, ventas por publicacion y datos de facturacion. Esta solución se intentó realizar pero al momento de realizar la consulta
la API devuelve el resultado vacio. 


In [2]:
# Importo librerias
import requests as r
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

## CREACION DEL DF CATEGORIAS
### Contiene los datos de todas las categorias disponibles en Mercado Libre Argentina

In [12]:
url = "https://api.mercadolibre.com/sites/MLA/categories"
request = r.get(url)
results = request.json()
categorias = pd.DataFrame(results)

#### Al analizar algunos items de las categorias, se observó que en las siguientes no siempre cuentan con información acerca de reputacion
#### Se separarán de este análisis inicial de clasificación de clientes, se propondrá en una siguiente iteración del proyecto abarcar estas categorias.  

In [13]:
categorias_out = categorias.loc[(categorias.id == 'MLA1743') | (categorias.id == 'MLA2547') | (categorias.id == 'MLA1459') | (categorias.id == 'MLA1540') | (categorias.id == 'MLA1953')].reset_index(drop=True)

In [14]:
categorias = categorias.loc[~categorias.id.isin(categorias_out.id),:].reset_index(drop=True)
categorias.rename(columns={'id':'categoria_MLA', 'name':'categoria_name'},inplace=True)
categorias

,categoria_MLA,categoria_name
0,MLA5725,Accesorios para Vehículos
1,MLA1512,Agro
2,MLA1403,Alimentos y Bebidas
3,MLA1071,Animales y Mascotas
4,MLA1367,Antigüedades y Colecciones
5,MLA1368,"Arte, Librería y Mercería"
6,MLA1384,Bebés
7,MLA1246,Belleza y Cuidado Personal
8,MLA1039,Cámaras y Accesorios
9,MLA1051,Celulares y Teléfonos


In [22]:
categorias.to_csv('../data/categorias.csv')

In [15]:
# se crea la lista de offset para iterar y obtener los primeros 1.000 items de cada categoria seleccionada
offset_list = np.arange(0, 1000, 50)

## CREACION DEL DATASET DE ITEMS
### Contiene los primeros 1.000 items de cada categoria

In [ ]:
# creo el dataset de items, en busca de obtener los ID de los sellers
# se crean las primeras variables consideradas relevantes para el análisis 
# estas variables se encuentran relacionadas con su id, fecha de registro y reputacion

df_items = pd.DataFrame()
idx = 0

for idx in range(len(categorias)):

    categoria = categorias.categoria_MLA[idx]
    
    print("proceso la categoria:", categoria)

    for offset in offset_list:

        url = f'https://api.mercadolibre.com/sites/MLA/search?category={categoria}&offset={offset}'

        req = r.get(url)

        items = req.json()['results']

        for item in range(len(items)):

            aux = pd.DataFrame({
                "categoria_MLA" : categoria,
                "seller" : items[item]['seller']['id'],
                "fecha_registro" : items[item]['seller']['registration_date'],
                "power_seller_status" :items[item]['seller']['seller_reputation']['power_seller_status'],
                "level_id" :items[item]['seller']['seller_reputation']['level_id'],
                # las siguientes columnas refieren a los últimos 60 días de actividad
                "cancellations_value" :items[item]['seller']['seller_reputation']['metrics']['cancellations']['value'],
                "cancellations_rate" :items[item]['seller']['seller_reputation']['metrics']['cancellations']['rate'],
                "claims_value" :items[item]['seller']['seller_reputation']['metrics']['claims']['value'],
                "claims_rate" :items[item]['seller']['seller_reputation']['metrics']['claims']['rate'],
                "delayed_handling_time_value" :items[item]['seller']['seller_reputation']['metrics']['delayed_handling_time']['value'],
                "delayed_handling_time_rate" :items[item]['seller']['seller_reputation']['metrics']['delayed_handling_time']['rate'],
                "sales_value" :items[item]['seller']['seller_reputation']['metrics']['sales']['completed'],
                # datos históricos y reputación
                "transactions_total_historic": items[item]['seller']['seller_reputation']['transactions']['total'],
                "transactions_canceled_historic": items[item]['seller']['seller_reputation']['transactions']['canceled'],
                "transactions_completed_historic": items[item]['seller']['seller_reputation']['transactions']['completed'],
                "rating_negative": items[item]['seller']['seller_reputation']['transactions']['ratings']['negative'],
                "rating_neutral": items[item]['seller']['seller_reputation']['transactions']['ratings']['neutral'],
                "rating_positive": items[item]['seller']['seller_reputation']['transactions']['ratings']['positive'],
                # datos sobre el precio y ventas
                "price": items[item]['price'],
                "sold_quantity": items[item]['sold_quantity'],
                # datos de localidad y provincia
                "state":  items[item]['seller_address']['state']['name'],
                "city": items[item]['seller_address']['city']['name']
            }, index = {idx})
            df_items = df_items.append(aux)
            idx += 1
        print("termino el offset:", offset)
print("termino la categoria:", categoria)

In [217]:
print("cantidad de items importados: ", len(df_items))

cantidad de items importados:  26878


In [218]:
# de cada categoria se obtuvo la siguiente cantidad de items
df_items.value_counts('categoria_MLA')

categoria_MLA
MLA9304      1000
MLA1384      1000
MLA3937      1000
MLA1132      1000
MLA1512      1000
MLA1168      1000
MLA1500      1000
MLA1499      1000
MLA1430      1000
MLA1367      1000
MLA1039      1000
MLA407134     999
MLA1403       999
MLA1368       999
MLA1276       999
MLA1246       999
MLA1182       999
MLA1071       999
MLA1144       998
MLA1051       998
MLA5726       998
MLA1000       998
MLA1648       997
MLA409431     995
MLA5725       978
MLA3025       962
MLA1574       961
dtype: int64

In [4]:
# defino el día de la consulta para extraer el csv
from datetime import datetime
day = datetime.today().strftime('%Y-%m-%d').replace('-','')
day

'20220324'

In [7]:
# guardo los datos por items
df_items.to_csv(f'../data/{day}_dataset_items.csv',index=False)

## FUNCIONALIDAD NO IMPLEMENTADA

### IDEA DE EXTRACCION DE ITEMS POR SELLER
#### Se obtendrán los items activos de los sellers obtenidos en la consulta anterior
#### Se trabajará con el offset, para traer los primeros 1.000 registros
#### Se busca obtener datos de sus items, items vendidos, precios y de esta forma obtener una metrica aproximada de facturación y rentabilidad para cada seller

In [ ]:
df_sellers_test = df_sellers.head(5).reset_index(drop=True)
df_sellers_test

In [ ]:
# creo el dataset de items, en base a los sellers ID y la categoria de analisis

df_itemsSeller = pd.DataFrame()
idx = 0

for idx in range(len(df_sellers_test)):

    seller = df_sellers_test.seller[idx]
    categoria = df_sellers_test.categoria_MLA[idx]

    params = {
        'seller_id': {seller},
        'category': {categoria},
        'offset' : "50"
    }   

    url = f'https://api.mercadolibre.com/sites/MLA/search?

    req = r.get(url, params = params)
    
    print(url)

    items = req.json()

    for item in range(len(items)):

        aux = pd.DataFrame({
            "categoria_MLA" : categoria,
            "seller" : items[item]['seller']['id'],
                }, index = {idx})
        df_itemsSeller = df_itemsSeller.append(aux)
        idx += 1

In [ ]:
items